## Author: Blaine Hill

In this notebook, we program out how to embed a KG such as FB15k_237 using RotatE. The weights are saved under embedding_model_weights.pth

In [1]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "embedding_model.ipynb"

import wandb
wandb.login()

import torch
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import RotatE
from torch_geometric.loader import DataLoader
import torch.optim as optim

import os
import os.path as osp

from datetime import datetime
from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import ComplEx

import yaml
from icecream import ic

wandb: Currently logged in as: bthill1. Use `wandb login --relogin` to force relogin


In [2]:
dataset_name='FB15k_237'
embedding_model_name='RotatE'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = osp.join(os.getcwd(), '..', 'data', dataset_name)

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

/home/blaineh2/anaconda3/envs/ScoreMatchingDiffKG/lib/python3.9/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def build_model(config):

    model = RotatE(num_nodes=train_data.num_nodes, num_relations=train_data.num_edge_types, hidden_channels=config.hidden_channels).to(device) 

    train_loader = model.loader(
        head_index=train_data.edge_index[0],
        rel_type=train_data.edge_type,
        tail_index=train_data.edge_index[1],
        batch_size=config.batch_size,
        shuffle=True,
    )
    val_loader = model.loader(
        head_index=val_data.edge_index[0],
        rel_type=val_data.edge_type,
        tail_index=val_data.edge_index[1],
        batch_size=config.batch_size,
        shuffle=True,
    )
    test_loader = model.loader(
        head_index=test_data.edge_index[0],
        rel_type=test_data.edge_type,
        tail_index=test_data.edge_index[1],
        batch_size=config.batch_size,
        shuffle=True,
    )
    
    optimizer = optim.Adagrad(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

    
    return model, optimizer, train_loader, val_loader, test_loader

In [4]:
def train_model(model, optimizer, train_loader):
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in train_loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        #scale on specific number of elements in batch
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test_model(model, config, test_loader):
    model.eval()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in test_loader:
        loss = model.loss(head_index, rel_type, tail_index)
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()

    mean_rank, mrr, hits_at_k = model.test(
        head_index=head_index,
        rel_type=rel_type,
        tail_index=tail_index,
        batch_size=config.batch_size,
        k=config.k,
    )

    return total_loss / total_examples, mean_rank, mrr, hits_at_k

In [5]:
def main(config=None, verbose=False):
    with wandb.init(
        project=f"ScoreMatchingDiffKG_Embedding",
        name=f"{dataset_name}_{embedding_model_name}_embedding_model {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        config=config if config is not None else {}
    ):
        config = wandb.config
        model, optimizer, train_loader, val_loader, test_loader = build_model(config)
        wandb.watch(model)
        for epoch in range(config.epochs):
            loss = train_model(model, optimizer, train_loader)
            if verbose: print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}')
            train_metrics ={
                "train_epoch": epoch,
                "train_loss": loss
            }
            if epoch % 25 == 0 and epoch > 0:
                loss, mean_rank, mrr, hits_at_10 = test_model(model, config, val_loader)
                if verbose: print(f'Val Mean Rank: {mean_rank:.2f}, Val Mean Reciprocal Rank: {mrr:.2f}, Val Hits@10: {hits_at_10:.4f}')
                val_metrics ={
                    "val_loss": loss,
                    "val_mean_rank": mean_rank,
                    "val_mrr": mrr,
                    "val_hits_at_10": hits_at_10
                }
            #log to wandb
            wandb.log({**train_metrics, **val_metrics} if 'val_metrics' in locals() else {**train_metrics})

        #once everything is finished, test model
        loss, mean_rank, mrr, hits_at_10 = test_model(model, config, test_loader)
        if verbose: print(f'Test Mean Rank: {mean_rank:.2f}, Test Mean Reciprocal Rank: {mrr:.2f}, Test Hits@10: {hits_at_10:.4f}')
        test_metrics ={
                    "test_loss": loss,
                    "test_mean_rank": mean_rank,
                    "test_mrr": mrr,
                    "test_hits_at_10": hits_at_10
                }
        wandb.log({**test_metrics})

        # Save the trained model
        path = osp.join(os.getcwd(), f'{dataset_name}_{embedding_model_name}_embedding_model_weights.pth')
        torch.save(model.state_dict(), path)

        # Fetch a batch from train_loader
        for batch in train_loader:
            # Assuming batch contains head_index, rel_type, tail_index, and possibly other data
            head_index, rel_type, tail_index = batch
            break  # Only need one batch for this purpose

        # Use the fetched batch to provide dummy inputs for the export
        # Ensure these variables are moved to the same device as model if necessary
        torch.onnx.export(
            model,
            (head_index, rel_type, tail_index),  # Use actual data as dummy inputs
            f'{dataset_name}_{embedding_model_name}_embedding_model_weights.onnx',
            opset_version=11,
            do_constant_folding=True,
            input_names=["head_index", "rel_type", "tail_index"],  # Adjust input names as needed
            dynamic_axes={
                "head_index": {0: "batch_size"}, 
                "rel_type": {0: "batch_size"}, 
                "tail_index": {0: "batch_size"}
            }
        )
        wandb.save(f'{dataset_name}_{embedding_model_name}_embedding_model_weights.onnx')

        return model
    

In [6]:
run_sweep = True
if run_sweep:
    with open('sweep_config.yaml', 'r') as file:
        sweep_config = yaml.safe_load(file)
else:
    config = {
            "epochs": 500,
            "batch_size": 1000,
            "lr": 0.001,
            "weight_decay": 1e-6,
            "k": 10, #used for top-k evaluation
            "hidden_channels": 50
        }

In [7]:
if run_sweep:
    
    with open('sweep_config.yaml', 'r') as file:
        sweep_config = yaml.safe_load(file)

    sweep_id = wandb.sweep(project=f"ScoreMatchingDiffKG_Embedding_Sweep",sweep=sweep_config)
    
    wandb.agent(sweep_id, function=main)
else:
    model = main(config, verbose=True)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: wfe3hq9n
Sweep URL: https://wandb.ai/bthill1/ScoreMatchingDiffKG_Embedding_Sweep/sweeps/wfe3hq9n


wandb: Agent Starting Run: 3crvj5js with config:
wandb: 	batch_size: 500
wandb: 	epochs: 1000
wandb: 	hidden_channels: 100
wandb: 	k: 10
wandb: 	lr: 1.003668058140734
wandb: 	weight_decay: 1e-06
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


ic| config: None
ic| wandb.config: {'batch_size': 500, 'epochs': 1000, 'hidden_channels': 100, 'k': 10, 'lr': 1.003668058140734, 'weight_decay': 1e-06}
